# Évaluation des intéractions entre le véhicule et son environnement

In [1]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes
nusc = NuScenes(version='v1.0-mini', dataroot='/Users/tiffanydarini/nuscenes-devkit/python-sdk/tutorials/data/sets/nuscenes/v1.0-mini', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.345 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


### Étape 1 : Récupération des données sur le véhicule ego

In [2]:
# Fonction qui renvoie les tables ego_pose d'une scène donnée
def get_poses(scene_token):
    pose_list = []
    scene_rec = nusc.get('scene', scene_token)
    sample_rec = nusc.get('sample', scene_rec['first_sample_token'])
    sd_rec = nusc.get('sample_data', sample_rec['data']['LIDAR_TOP'])
    
    ego_pose = nusc.get('ego_pose', sd_rec['token'])
    pose_list.append(ego_pose)

    while sd_rec['next'] != '':
        sd_rec = nusc.get('sample_data', sd_rec['next'])
        ego_pose = nusc.get('ego_pose', sd_rec['token'])
        pose_list.append(ego_pose)

    return pose_list

In [3]:
# On utilise ce qu'on a déjà (liste ego_poses)
scene_name = 'scene-0061'
scene_token = nusc.field2token('scene','name',scene_name)[0]
Liste_ego_poses = get_poses(scene_token) 

# Création des liste de coordonnées du véhicule égo 
X_coord =[]
Y_coord =[]
Z_coord =[]
T_timestamp = [] # liste des timestamps
for k in range(0,len(Liste_ego_poses)):
    X_coord.append(Liste_ego_poses[k]['translation'][0])
    Y_coord.append(Liste_ego_poses[k]['translation'][1])
    Z_coord.append(Liste_ego_poses[k]['translation'][2])
    T_timestamp.append(Liste_ego_poses[k]['timestamp'])

### Étape 2 : On récupère les échantillons de la scène 

In [4]:
# Ensemble des token des échantillons de la scene 
sample_token = nusc.field2token('sample','scene_token',scene_token)
len(sample_token)

39

### Étape 3 : On récupère les sample_annotation de la scène (boites d'objets dans la scène)

In [5]:
# Ensemble des sample_annotation de la scène :

sample_annotation_token = []
for k in sample_token : 
    L = nusc.field2token('sample_annotation','sample_token',k)
    sample_annotation_token = sample_annotation_token + L

### Étape 4 : Liste des instances dans la scène

In [6]:
# On créé la liste des token des instances présents dans les sample_annotation
L_instance_token = []
for k in sample_annotation_token :
    exemple = nusc.get('sample_annotation', k)
    L_instance_token.append(exemple['instance_token'])

In [7]:
# Puisque pour un même objet, on peut avoir plusieurs annotations, on supprime les doublons de L
L_instance_token = list(set(L_instance_token))
len(L_instance_token)
print('Il y a '+str(len(L_instance_token))+' instances dans la scène ' +scene_name)

Il y a 227 instances dans la scène scene-0061


### Étape 5 : Séparation de chaque catégorie 

In [8]:
# Liste des catégories d'objet dans la scène 
L_category_name = []
for k in sample_annotation_token :
    exemple = nusc.get('sample_annotation', k)
    L_category_name.append(exemple['category_name'])
L_category_name = list(set(L_category_name))
L_category_name

['vehicle.truck',
 'movable_object.barrier',
 'movable_object.trafficcone',
 'vehicle.bus.rigid',
 'movable_object.pushable_pullable',
 'human.pedestrian.adult',
 'vehicle.bicycle',
 'vehicle.car',
 'movable_object.debris',
 'human.pedestrian.construction_worker',
 'vehicle.motorcycle',
 'vehicle.construction']

In [9]:
# Pour l'instant, travail à la main pour chaque catégorie trouvée
k = 7
catego_name = L_category_name[k]
catego_token = nusc.field2token('category','name',catego_name)
category = nusc.get('category',catego_token[0])
category

{'token': 'fd69059b62a3469fbaef25340c0eab7f',
 'name': 'vehicle.car',
 'description': 'Vehicle designed primarily for personal use, e.g. sedans, hatch-backs, wagons, vans, mini-vans, SUVs and jeeps. If the vehicle is designed to carry more than 10 people use vehicle.bus. If it is primarily designed to haul cargo use vehicle.truck. '}

In [10]:
# Ensemble des instances appartenant à la fois à la scène et à la catégorie
L_instance_category = nusc.field2token('instance','category_token',catego_token[0]) # instance dans la catégorie
L_instance_category = [x for x in L_instance_category if x in L_instance_token]# et dans la scène 

In [11]:
L=[]
for k in L_instance_category : 
    instance_category = nusc.get('instance',k)
    nbr_annotations = instance_category['nbr_annotations']
    if nbr_annotations == 1:
        L.append([instance_category['first_annotation_token']])
    else : 
        H = []
        first_token = instance_category['first_annotation_token']
        last_token = instance_category['last_annotation_token']
        current_token = first_token 
        
        i = 0 
        while i!= nbr_annotations :
            current_ann = nusc.get('sample_annotation', current_token)
            H.append(current_ann['token'])
            current_token = current_ann['next'] #On passe au suivant
            i+=1
        L.append(H)
# Il peut y avoir plusieurs éléments de la catégorie dans la scène, ici chaque ligne de L représente 
# les token des sample_annotation pour un élément de cette catégorie. 

In [12]:
# Maintenant qu'on a les token de tous les sample_annotation pour tous les éléments d'une même catégorie, 
# on va pouvoir récupérer leurs positions et tracer leurs trajectoires

X = []
Y = []
Z = []
T_time = []
for k in L :
    x = []
    y = []
    z = []
    t = []
    for j in k :
        sample_annotation = nusc.get('sample_annotation',j)
        x.append(sample_annotation['translation'][0])
        y.append(sample_annotation['translation'][1])
        z.append(sample_annotation['translation'][2])
        t.append(nusc.get('sample',sample_annotation['sample_token'])['timestamp'])
    X.append(x)
    Y.append(y)
    Z.append(z)
    T_time.append(t)

### Étape 6 : Fonction pour savoir si un objet est immobile ou pas 

In [13]:
# On crée une fonction qui renvoie la liste des rangs dans les listes X,Y des objets immobiles de 
# cette liste et donc d'une même catégorie, ainsi que la liste des objets mobiles 
from math import sqrt

def immobile(X,Y):
    Immo = []
    Mobile = []
    n = len(X) # Nombre d'élément dans la catégorie 
    for i in range(0,n-1):
        X_coord = X[i]
        Y_coord = Y[i]
        if len(X_coord)==1 :
            Immo.append(i)
        else : 
            d = 0 
            for k in range(0,len(X_coord)-1):
                d+= sqrt((X_coord[k+1]-X_coord[k])**2+(Y_coord[k+1]-Y_coord[k])**2)
            if d<2 : # Si la distance totale parcourue par l'objet est inférieur à 2m, on considère que l'objet bouge pas
                Immo.append(i)
            else :
                Mobile.append(i)
    return Immo,Mobile  

In [14]:
Immo, Mobile = immobile(X,Y)
len(Mobile)

5

### Étape 7 : On garde uniquement les objets mobiles

In [15]:
X_mobile=[X[i] for i in Mobile]
Y_mobile=[Y[i] for i in Mobile]
Z_mobile=[Z[i] for i in Mobile]
T_mobile = [T_time[i] for i in Mobile]
len(X_mobile)

5

### Étape 8 : Création des vecteurs

In [16]:
import numpy as np
import datetime 
from datetime import timedelta

In [17]:
# On commence par le véhicule ego 
V_ego = []
for k in range(0,len(X_coord)-1):
    t2 = datetime.datetime.fromtimestamp(T_timestamp[k+1]/ 1e6)
    t1 = datetime.datetime.fromtimestamp(T_timestamp[k]/ 1e6)
    t = (t2-t1).total_seconds()
    v = np.array([X_coord[k+1]-X_coord[k],Y_coord[k+1]-Y_coord[k]])/t # Le vecteur a pour norme la vitesse à l'instant k
    V_ego.append(v)

In [18]:
# On va travailler avec une df
import pandas as pd

In [19]:
df_vecteurs = pd.DataFrame({'V_ego':V_ego})
df_vecteurs

,V_ego
0,"[-3.1712023320455462, -8.567313420091782]"
1,"[-3.1207235773484103, -8.562076268008871]"
2,"[-3.1653930094341445, -8.53617857224011]"
3,"[-3.1740825254365994, -8.49344994582648]"
4,"[-3.243236261012891, -8.504405724567667]"
...,...
376,"[1.9723523754827463, -0.4720986514141277]"
377,"[1.783906470998763, -0.4205889639945817]"
378,"[1.8364963045822984, -0.414838869174028]"
379,"[1.9999102240345528, -0.44535267086528335]"


In [20]:
# On passe aux éléments de la catégorie 
i = 0 
for j in range(0,len(X_mobile)) :
    i = i+ 1
    V = []
    for k in range(0,len(X_mobile[j])-1):
        t2 = datetime.datetime.fromtimestamp(T_mobile[j][k+1]/ 1e6)
        t1 = datetime.datetime.fromtimestamp(T_mobile[j][k]/ 1e6)
        t = (t2-t1).total_seconds()
        v = np.array([X_mobile[j][k+1]-X_mobile[j][k],Y_mobile[j][k+1]-Y_mobile[j][k]])
        V.append(v)
    # On a pas autant de vecteur pour un objet que pour le véhicule ego
    nbr_case_vide = len(V_ego)-len(V)
    Complement = [np.nan]*nbr_case_vide # on va donc compléter par des cases vides pour mettre dans la df
    Vf = V + Complement
    df_vecteurs[catego_name+ str(i)]=Vf

In [21]:
df_vecteurs

,V_ego,vehicle.car1,vehicle.car2,vehicle.car3,vehicle.car4,vehicle.car5
0,"[-3.1712023320455462, -8.567313420091782]","[1.9870000000000232, 4.3539999999998145]","[-0.41200000000003456, -0.7470000000000709]","[-2.197999999999979, -5.18100000000004]","[-0.8670000000000186, -2.4420000000000073]","[-1.5790000000000077, -3.6520000000000437]"
1,"[-3.1207235773484103, -8.562076268008871]","[2.1499999999999773, 4.671000000000049]","[-0.10899999999998045, -0.2179999999998472]","[-2.4879999999999995, -5.692999999999984]","[-0.8799999999999955, -2.5]","[-1.5790000000000077, -3.6519999999998163]"
2,"[-3.1653930094341445, -8.53617857224011]",NaN,"[-0.19499999999999318, -0.9660000000001219]","[-2.009999999999991, -5.037000000000035]","[-0.4919999999999618, -2.310000000000173]","[-1.5790000000000077, -3.6520000000000437]"
3,"[-3.1740825254365994, -8.49344994582648]",NaN,"[-0.5600000000000023, -1.2539999999999054]","[-2.2410000000000423, -4.997000000000071]","[-0.4910000000000423, -2.269999999999982]","[-1.5779999999999745, -3.6520000000000437]"
4,"[-3.243236261012891, -8.504405724567667]",NaN,"[-0.7340000000000373, -1.7799999999999727]",NaN,"[-0.33899999999999864, -1.90300000000002]","[-1.910000000000025, -4.276000000000067]"
...,...,...,...,...,...,...
376,"[1.9723523754827463, -0.4720986514141277]",NaN,NaN,NaN,NaN,NaN
377,"[1.783906470998763, -0.4205889639945817]",NaN,NaN,NaN,NaN,NaN
378,"[1.8364963045822984, -0.414838869174028]",NaN,NaN,NaN,NaN,NaN
379,"[1.9999102240345528, -0.44535267086528335]",NaN,NaN,NaN,NaN,NaN
